### Heterogeneous Vehicle Routing Problem (HEVRP) usando Gurobi en Python: `gurobipy`

---
---



### Generar datos aleatorios

---

##### Especificando el número de nodos podemos generar aleteatoriamente una matriz de distancias en un espacio $[0,1]^2$. Además, especificando la máxima demanda que podria haber en cada nodo podemos asignar aleatoriamente la demanda.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import gurobipy

rnd       = np.random.RandomState(seed= 17)

max_capacidad = 30
n_vehiculos       = 3
max_demanda  = 10
n_nodos            = 10

locaciones       = rnd.uniform(0,1,size=(n_nodos, 2))
demandas        = list( rnd.randint(1,max_demanda, n_nodos) )
demandas[-1]  = 0 # demanda del deposito
id_nodos          = [i for i in range(0, n_nodos)]
id_clientes       = [i for i in range(0, n_nodos - 1)]
id_vehiculos    = [i for i in range(0,n_vehiculos)]

demanda_total     = np.sum(demandas)
capacidades          = np.sort( list(rnd.randint(1, max_capacidad, n_vehiculos)))

matriz_distancias = euclidean_distances(locaciones)*10e4

##### Creamos el dictionario `q` con keys son los  ids del nodo y elemento es la demanda.


In [ ]:
q     = { i: demandas[i] for i in id_nodos}

##### `gurobipy` tiene funciones que puede automatizar la creación de variables usando `tuplelist` y `tupledict`. De esta manera creamos el diccionario `distancias_dic` donde cada `key` es la tupla `(i,j)`

In [ ]:
arcos = {(i,j) for i in nodos for j in nodos if i!=j}

distancias_dic = {}
for i,j in arcos:
    if i!=j:
        distancias_dic[(i,j)] = matriz_distancias[i,j]
    
arcos_tl, distancias_tl = gurobipy.multidict(distancias_dic) #donde arcos_tl es un tuplelist y distancias_tl es un tupledict

### Especificamos modelo

---

##### La formulación según detallado en (Gheysen, Golden & Assad, 1984) es la siguiente:
##### Se consideran $n$ clientes donde el depósito se encuentra en la locación $n-1$ e indexadas con $i = 1,...., n$
##### Los siguientes entradas se asumen disponibles:
##### T = numeros de vehiculos
##### $Q_k$ = capacidad del vehículo $k$; ($Q_1 < Q_2 <....<Q_T$)
##### $q_j$ = demanda del cliente $j$
##### $d_{ij}$ = distancia de viaje entre cliente $i$ y cliente $j$.

##### Creamos el modelo de optimización de `gurobipy` llamado "VRP" y la almacenamos en la variable `m`.

In [ ]:
m = gurobipy.Model('VRP')

###### **Opcional:** Podemos especificar los cores que queremos usar para resolver el problema

In [ ]:
m.Params.Threads    = 8

### Especificamos las variables

---

#### *Variable de si vehiculo $k$ transita arco $i,j$:*
#### $x^k_{i,j} = \begin{cases} 1 & \text{si vehiculo } k \text{ viaja desde } i \text{ a } j \\ 0 & \text{en otro caso} \end{cases}$
#### *Variable de carga transportada en el arco $i,j$:*
#### $u_{i,j}$ = la carga transportada de $i$ a $j$

In [ ]:
# Variables
x_1 = m.addVars(arcos_tl, name="x_1", vtype=gurobipy.GRB.BINARY)
x_2 = m.addVars(arcos_tl, name="x_2", vtype=gurobipy.GRB.BINARY)
x_3 = m.addVars(arcos_tl, name="x_3", vtype=gurobipy.GRB.BINARY)
u   = m.addVars(arcos_tl, name="u")

### Especificamos función objetivo

---
##### $$\min \sum_{i=1}^{n} \sum_{j=1}^{n} \sum_{k=1}^T d_{i,j} \cdot  x^k_{i,j}$$

In [ ]:
obj  =  gurobipy.LinExpr()
obj += gurobipy.quicksum([matrix_distances[i,j]*x_1[i,j] for i,j in arcos]) 
obj += gurobipy.quicksum([matrix_distances[i,j]*x_2[i,j] for i,j in arcos]) 
obj += gurobipy.quicksum([matrix_distances[i,j]*x_3[i,j] for i,j in arcos]) 

m.setObjective(obj, gurobipy.GRB.MINIMIZE)

### Especificamos las restricciones

---

##### $1^{ra}$ restricción
##### $\sum_{k=1}^T \sum_{i=1}^n x_{i,j}^k=1\hspace{1cm} j=1,...,n$

In [ ]:
m.addConstrs(  (x_1.sum('*',j) + x_2.sum('*',j) +x_3.sum('*',j)  == 1 for j in id_clientes), "1ra")

##### $2^{da}$ restricción
##### $\sum_{i=1}^n x_{i,j}^k - \sum_{l=1}^nx_{j,l}^k = 0  \hspace{1cm} j=1,...,n; \hspace{0.2cm} k=1,...,T$

In [ ]:
m.addConstrs( (x_1.sum('*',j) - x_1.sum(j,'*') == 0 for j in id_nodos), "2da_1")
m.addConstrs( (x_2.sum('*',j) - x_2.sum(j,'*') == 0 for j in id_nodos), "2da_2")
m.addConstrs( (x_3.sum('*',j) - x_3.sum(j,'*') == 0 for j in id_nodos), "2da_3")

##### $3^{ra}$ restricción
##### $\sum_{i=1}^n u_{i,j} - \sum_{l=1}^n u_{j,l}= q_j  \hspace{1cm} j=1,...,n$

In [ ]:
m.addConstrs( (u.sum('*',j) - u.sum(j,'*') == q[j] for j in id_clientes), "3ra")

##### $4^{ta}$ restricción
##### $u_{n-1,j}\leq \sum_{k=1}^T Q_k \cdot x_{n-1,j}^k   \hspace{1cm} j=1,...,n$

In [ ]:
m.addConstrs( (u[n_nodos-1,j] <= capacidades[0]*x_1[n_nodos-1,j] + capacidades[1]*x_2[n_nodos-1,j]+ capacidades[2]*x_3[n_nodos-1,j] for j in id_clientes), "4ta")

##### $5^{ta}$ restricción
##### $u_{i,j}\leq M \cdot \sum_{k=1}^T  x_{i,j}^k   \hspace{1cm} i \neq j=1,...,n$

In [ ]:
m.addConstrs( (u[i,j] <= 10000*(x_1[i,j]+x_2[i,j]+x_3[i,j]) for i,j in arcos_tl), "5ta")

### Resolvemos

---

In [ ]:
# Solve
m.optimize()

### Mostramos resultados

---

In [ ]:
for v in m.getVars():
    if v.x > 0 :
        print('%s %g' % (v.varName, v.x))